In [1]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

In [ ]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 51.5,
	"longitude": 4.5,
	"start_date": "2021-01-01",
	"end_date": "2021-12-31",
	"daily": ["temperature_2m_mean", "temperature_2m_max", "temperature_2m_min", "daylight_duration", "sunshine_duration", "sunset", "sunrise", "weather_code", "shortwave_radiation_sum", "apparent_temperature_min", "apparent_temperature_mean", "apparent_temperature_max"],
	"hourly": ["temperature_2m", "soil_temperature_0_to_7cm", "weather_code", "shortwave_radiation_instant", "cloud_cover", "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high"],
	"timezone": "Europe/Berlin",
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone: {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

Coordinates: 51.9156379699707°N 4.450549602508545°E
Elevation: 2.0 m asl
Timezone: b'Europe/Berlin'b'GMT+2'
Timezone difference to GMT+0: 7200s


In [16]:
# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_soil_temperature_0_to_7cm = hourly.Variables(1).ValuesAsNumpy()
hourly_weather_code = hourly.Variables(2).ValuesAsNumpy()
hourly_shortwave_radiation_instant = hourly.Variables(3).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(4).ValuesAsNumpy()
hourly_cloud_cover_low = hourly.Variables(5).ValuesAsNumpy()
hourly_cloud_cover_mid = hourly.Variables(6).ValuesAsNumpy()
hourly_cloud_cover_high = hourly.Variables(7).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["soil_temperature_0_to_7cm"] = hourly_soil_temperature_0_to_7cm
hourly_data["weather_code"] = hourly_weather_code
hourly_data["shortwave_radiation_instant"] = hourly_shortwave_radiation_instant
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["cloud_cover_low"] = hourly_cloud_cover_low
hourly_data["cloud_cover_mid"] = hourly_cloud_cover_mid
hourly_data["cloud_cover_high"] = hourly_cloud_cover_high

hourly_dataframe = pd.DataFrame(data = hourly_data)
hourly_dataframe

,date,temperature_2m,soil_temperature_0_to_7cm,weather_code,shortwave_radiation_instant,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high
0,2020-12-31 22:00:00+00:00,-1.032500,0.867500,1.0,0.0,34.0,34.0,0.0,0.0
1,2020-12-31 23:00:00+00:00,-1.132500,0.817500,2.0,0.0,54.0,53.0,0.0,2.0
2,2021-01-01 00:00:00+00:00,-0.132500,1.717500,3.0,0.0,97.0,32.0,9.0,0.0
3,2021-01-01 01:00:00+00:00,0.417500,1.517500,0.0,0.0,13.0,13.0,2.0,0.0
4,2021-01-01 02:00:00+00:00,0.167500,1.167500,1.0,0.0,35.0,35.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...
8755,2021-12-31 17:00:00+00:00,10.767500,10.317500,2.0,0.0,70.0,14.0,0.0,65.0
8756,2021-12-31 18:00:00+00:00,10.767500,10.117499,3.0,0.0,97.0,25.0,0.0,96.0
8757,2021-12-31 19:00:00+00:00,10.767500,9.967500,3.0,0.0,80.0,30.0,0.0,71.0
8758,2021-12-31 20:00:00+00:00,10.717500,9.917500,3.0,0.0,100.0,100.0,1.0,94.0


In [17]:
hourly_dataframe.to_csv("hourly_data_2021.csv", index = False)

In [18]:
# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()
daily_daylight_duration = daily.Variables(3).ValuesAsNumpy()
daily_sunshine_duration = daily.Variables(4).ValuesAsNumpy()
daily_sunset = daily.Variables(5).ValuesInt64AsNumpy()
daily_sunrise = daily.Variables(6).ValuesInt64AsNumpy()
daily_weather_code = daily.Variables(7).ValuesAsNumpy()
daily_shortwave_radiation_sum = daily.Variables(8).ValuesAsNumpy()
daily_apparent_temperature_min = daily.Variables(9).ValuesAsNumpy()
daily_apparent_temperature_mean = daily.Variables(10).ValuesAsNumpy()
daily_apparent_temperature_max = daily.Variables(11).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}

daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["daylight_duration"] = daily_daylight_duration
daily_data["sunshine_duration"] = daily_sunshine_duration
daily_data["sunset"] = daily_sunset
daily_data["sunrise"] = daily_sunrise
daily_data["weather_code"] = daily_weather_code
daily_data["shortwave_radiation_sum"] = daily_shortwave_radiation_sum
daily_data["apparent_temperature_min"] = daily_apparent_temperature_min
daily_data["apparent_temperature_mean"] = daily_apparent_temperature_mean
daily_data["apparent_temperature_max"] = daily_apparent_temperature_max

daily_dataframe = pd.DataFrame(data = daily_data)

In [19]:
daily_dataframe.to_csv("daily_data_2021.csv", index = False)